In [9]:
import requests
import os
import json
import pandas as pd
import re
from collections import defaultdict
import time
from pathlib import Path
from requests.exceptions import RequestException
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.cm as cm
from matplotlib.colors import Normalize
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns



In [2]:
# Step 1: Create folder to store data
os.makedirs('data', exist_ok=True)

# Step 2: Get the initial data with error handling
try:
    response = requests.get("https://2025electionresults.comelec.gov.ph/data/regions/local/5532000.json")
    response.raise_for_status()  # Raise exception for non-200 status codes
    data = response.json()
except RequestException as e:
    print(f"Failed to get initial data: {e}")
    exit(1)
except json.JSONDecodeError:
    print("Failed to decode initial JSON data")
    exit(1)

# Step 3: Extract all the codes from the regions with validation
if "regions" not in data:
    print("Expected 'regions' key not found in data")
    exit(1)

codes = [region.get('code') for region in data["regions"] if region.get('code')]
print(f"Found {len(codes)} region codes")

# Step 4: Loop through the codes and fetch the data
for code in codes:
    try:
        url = f"https://2025electionresults.comelec.gov.ph/data/regions/precinct/55/{code}.json"
        print(f"Fetching barangay data from: {url}")
        response_barangay = requests.get(url)
        response_barangay.raise_for_status()
        data_barangay = response_barangay.json()
        
        # Add delay to prevent rate limiting
        time.sleep(1)
        
        # Extract the 'code' from the new data with validation
        if "regions" not in data_barangay:
            print(f"No 'regions' key found for code {code}")
            continue
            
        barangay_codes = [barangay.get('code') for barangay in data_barangay["regions"] if barangay.get('code')]
        print(f"Found {len(barangay_codes)} barangay codes for region {code}")
        
        # Step 5: Loop through each barangay code and save the data
        for barangay_code in barangay_codes:
            # Try multiple URL patterns if the first one fails
            urls_to_try = [
                f"https://2025electionresults.comelec.gov.ph/data/er/553/{barangay_code}.json",
                f"https://2025electionresults.comelec.gov.ph/data/er/55/{barangay_code}.json",
                f"https://2025electionresults.comelec.gov.ph/data/er/{code}/{barangay_code}.json"
            ]
            
            success = False
            for url in urls_to_try:
                try:
                    print(f"Trying: {url}")
                    response_final = requests.get(url)
                    response_final.raise_for_status()
                    data_final = response_final.json()
                    
                    # Validate the expected data structure
                    if "information" not in data_final or "location" not in data_final["information"]:
                        print(f"Missing expected data structure in response from {url}")
                        continue
                    
                    # Extract location and get the last part (barangay name)
                    location = data_final["information"]["location"]
                    location_last_part = location.split(',')[-1].strip().lower().replace(" ", "-")
                    
                    # Save with barangay code to ensure uniqueness
                    filename = f'data/{location_last_part}_{barangay_code}.json'
                    counter = 2
                    while os.path.exists(filename):
                        filename = f'data/{location_last_part}_{barangay_code}_{counter}.json'
                        counter += 1
                    
                    with open(filename, 'w', encoding='utf-8') as file:
                        json.dump(data_final, file, ensure_ascii=False, indent=4)
                    
                    print(f"Saved: {filename}")
                    success = True
                    break  # Exit the URL loop if successful
                    
                except RequestException as e:
                    print(f"Failed to fetch from {url}: {e}")
                except json.JSONDecodeError:
                    print(f"Failed to decode JSON from {url}")
                    
            if not success:
                print(f"Failed to fetch data for barangay {barangay_code} after trying all URL patterns")
            
            # Add delay to prevent rate limiting
            time.sleep(1)
            
    except RequestException as e:
        print(f"Failed to fetch barangay data for region {code}: {e}")
        continue
    except json.JSONDecodeError:
        print(f"Failed to decode barangay JSON for region {code}")
        continue

Found 86 region codes
Fetching barangay data from: https://2025electionresults.comelec.gov.ph/data/regions/precinct/55/5532001.json
Found 2 barangay codes for region 5532001
Trying: https://2025electionresults.comelec.gov.ph/data/er/553/55320020.json
Saved: data/abanon_55320020.json
Trying: https://2025electionresults.comelec.gov.ph/data/er/553/55320021.json
Saved: data/abanon_55320021.json
Fetching barangay data from: https://2025electionresults.comelec.gov.ph/data/regions/precinct/55/5532002.json
Found 3 barangay codes for region 5532002
Trying: https://2025electionresults.comelec.gov.ph/data/er/553/55320022.json
Saved: data/agdao_55320022.json
Trying: https://2025electionresults.comelec.gov.ph/data/er/553/55320023.json
Saved: data/agdao_55320023.json
Trying: https://2025electionresults.comelec.gov.ph/data/er/553/55320024.json
Saved: data/agdao_55320024.json
Fetching barangay data from: https://2025electionresults.comelec.gov.ph/data/regions/precinct/55/5532003.json
Found 1 barangay 

In [3]:
def aggregate_barangay_data(data_folder):
    """
    Load and aggregate barangay JSON files with naming format <barangay>_<numeric code>.json
    """
    # Dictionary to hold aggregated data by barangay and candidate
    aggregated_votes = defaultdict(int)
    barangay_info = {}
    
    # Get all JSON files in the data folder
    json_files = [f for f in os.listdir(data_folder) if f.endswith('.json')]
    
    for file_name in json_files:
        file_path = os.path.join(data_folder, file_name)
        
        # Extract barangay name using a regular expression to handle numeric codes
        # This will match filenames like "abanon_55320020.json"
        match = re.match(r'^(.+?)_\d+\.json$', file_name)
        if match:
            base_barangay = match.group(1)
        else:
            # Fallback: just remove the .json extension
            base_barangay = file_name.replace('.json', '')
        
        # Read JSON data
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # The rest of the function remains the same
        # Store barangay information
        if base_barangay not in barangay_info:
            barangay_info[base_barangay] = {
                'location': data['information']['location'],
                'precinct_id': data['information']['precinctId'],
                'registered_voters': data['information']['numberOfRegisteredVoters'],
                'actual_voters': data['information']['numberOfActuallyVoters'],
                'turnout': data['information']['turnout']
            }
        
        # Process national contests
        for contest in data['national']:
            contest_name = contest['contestName']
            
            for candidate in contest['candidates']['candidates']:
                key = (base_barangay, 'National', contest_name, candidate['name'])
                aggregated_votes[key] += candidate['votes']
        
        # Process local contests
        for contest in data['local']:
            contest_name = contest['contestName']
            
            for candidate in contest['candidates']['candidates']:
                key = (base_barangay, 'Local', contest_name, candidate['name'])
                aggregated_votes[key] += candidate['votes']
    
    # Convert aggregated data to DataFrame format
    results = []
    for (barangay, contest_type, contest_name, candidate_name), votes in aggregated_votes.items():
        results.append({
            'barangay': barangay,
            'location': barangay_info[barangay]['location'],
            'precinct_id': barangay_info[barangay]['precinct_id'],
            'registered_voters': barangay_info[barangay]['registered_voters'],
            'actual_voters': barangay_info[barangay]['actual_voters'],
            'turnout': barangay_info[barangay]['turnout'],
            'contest_type': contest_type,
            'contest_name': contest_name,
            'candidate_name': candidate_name,
            'votes': votes
        })
    
    df = pd.DataFrame(results)
    
    # Calculate percentage for each candidate within their contest and barangay
    contest_total_votes = df.groupby(['barangay', 'contest_type', 'contest_name'])['votes'].transform('sum')
    df['percentage'] = (df['votes'] / contest_total_votes) * 100
    df['percentage'] = df['percentage'].round(2)
    
    return df

def main():
    # Replace with your data folder path
    data_folder = "data"
    
    # Aggregate barangay data
    df = aggregate_barangay_data(data_folder)
    
    # Display basic information
    print(f"Total candidates across all barangays: {len(df)}")
    print(f"Unique barangays: {df['barangay'].nunique()}")
    
    # Top candidates per contest (aggregated across all barangays)
    print("\nTop 5 candidates for Senator position:")
    senator_results = df[df['contest_name'] == 'SENATOR of PHILIPPINES']
    top_senators = senator_results.groupby('candidate_name')['votes'].sum().reset_index()
    top_senators = top_senators.sort_values('votes', ascending=False).head(5)
    print(top_senators)
    
    # Party List results
    print("\nTop 5 Party List results:")
    party_results = df[df['contest_name'] == 'PARTY LIST of PHILIPPINES']
    top_parties = party_results.groupby('candidate_name')['votes'].sum().reset_index()
    top_parties = top_parties.sort_values('votes', ascending=False).head(5)
    print(top_parties)
    
    # Save to CSV
    df.to_csv('aggregated_election_results.csv', index=False)
    print("\nAggregated data saved to 'aggregated_election_results.csv'")

if __name__ == "__main__":
    main()

Total candidates across all barangays: 22360
Unique barangays: 86

Top 5 candidates for Senator position:
              candidate_name  votes
44      5. AQUINO, BAM (KNP)  51346
54     59. SOTTO, TITO (NPC)  45514
59  63. TULFO, ERWIN (LAKAS)  44365
0    1. ABALOS, BENHUR (PFP)  41272
9     18. CAYETANO, PIA (NP)  40018

Top 5 Party List results:
             candidate_name  votes
79   3 FPJ PANDAY BAYANIHAN  17145
13                110 ABONO   9493
26        122 PINOY WORKERS   6406
6     104 SOLID NORTH PARTY   5479
103              51 AKBAYAN   5337

Aggregated data saved to 'aggregated_election_results.csv'


In [4]:
def create_contest_datasets(data_folder):
    """
    Create separate datasets for each contest with:
    - Candidates in rows (left side)
    - Barangays in columns (headers)
    - Vote counts in cells (aggregated across multiple files for the same barangay)
    
    Returns a tuple of (contest_datasets, all_barangays)
    """
    # Dictionary to store votes by contest, candidate, and barangay
    contest_data = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
    
    # List to track all barangays and contests we encounter
    all_barangays = set()
    all_contests = set()
    
    # Get all JSON files in the data folder
    json_files = [f for f in os.listdir(data_folder) if f.endswith('.json')]
    
    # Process each file
    for file_name in json_files:
        file_path = os.path.join(data_folder, file_name)
        
        # Extract base barangay name (remove _2, _3 suffixes if present)
        base_barangay = re.sub(r'_\d+\.json$', '', file_name)
        base_barangay = base_barangay.replace('.json', '')
        all_barangays.add(base_barangay)
        
        # Read JSON data
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Process national contests
        for contest in data['national']:
            contest_name = contest['contestName']
            all_contests.add(contest_name)
            
            # Process each candidate
            for candidate in contest['candidates']['candidates']:
                # Add votes for this candidate in this barangay
                # Using defaultdict(int) ensures we're adding to the existing count
                contest_data[contest_name][candidate['name']][base_barangay] += candidate['votes']
        
        # Process local contests
        for contest in data['local']:
            contest_name = contest['contestName']
            all_contests.add(contest_name)
            
            # Process each candidate
            for candidate in contest['candidates']['candidates']:
                # Add votes for this candidate in this barangay
                contest_data[contest_name][candidate['name']][base_barangay] += candidate['votes']
    
    # Convert to DataFrames
    dataframes = {}
    all_barangays_list = sorted(list(all_barangays))
    
    for contest_name in all_contests:
        # Create a list of dictionaries (one per candidate)
        rows = []
        for candidate_name, barangay_votes in contest_data[contest_name].items():
            # Start with the candidate name
            row = {'Candidate': candidate_name}
            
            # Add votes for each barangay
            for barangay in all_barangays_list:
                row[barangay] = barangay_votes.get(barangay, 0)
            
            # Add a total column
            row['Total'] = sum(barangay_votes.values())
            
            rows.append(row)
        
        # Create DataFrame and sort by total votes
        df = pd.DataFrame(rows)
        if not df.empty:
            df = df.sort_values('Total', ascending=False)
            
            # Move Candidate column to the front
            candidate_col = df.pop('Candidate')
            df.insert(0, 'Candidate', candidate_col)
        
        # Store in our dictionary of dataframes
        dataframes[contest_name] = df
    
    # Return both the dataframes and the list of barangays
    return dataframes, all_barangays_list

def main():
    # Replace with your data folder path
    data_folder = "data"
    
    # Create results folder if it doesn't exist
    results_folder = "./results-per-position"
    if not os.path.exists(results_folder):
        os.makedirs(results_folder)
        print(f"Created directory: {results_folder}")
    
    # Create contest datasets and get the list of barangays
    contest_datasets, all_barangays = create_contest_datasets(data_folder)
    
    # Display summary
    print(f"Created {len(contest_datasets)} contest datasets\n")
    
    # Display each dataset (or just the first few rows)
    for contest_name, df in contest_datasets.items():
        print(f"Dataset: {contest_name}")
        print(f"Shape: {df.shape[0]} candidates × {df.shape[1]} columns")
        print(df.head(5))  # Show top 5 candidates
        print("\n" + "="*50 + "\n")
        
        # Save to CSV in the results folder
        clean_name = re.sub(r'[^\w\s]', '', contest_name).replace(' ', '_')
        filename = os.path.join(results_folder, f"{clean_name}_results.csv")
        df.to_csv(filename, index=False)
        print(f"Saved to {filename}")
        print("\n")

    # Create a summary DataFrame of total votes per barangay
    barangay_summary = pd.DataFrame(index=all_barangays)
    for contest_name, df in contest_datasets.items():
        if 'Total' in df.columns:
            # Save the contest totals without the 'Candidate' column
            contest_cols = [col for col in df.columns if col not in ['Candidate', 'Total']]
            contest_totals = df[contest_cols].sum()
            barangay_summary[contest_name] = contest_totals
    
    # Add total votes column
    barangay_summary['Total Votes'] = barangay_summary.sum(axis=1)
    
    # Save summary to the results folder
    summary_filename = os.path.join(results_folder, 'barangay_vote_summary.csv')
    barangay_summary.to_csv(summary_filename)
    print(f"Saved barangay summary to '{summary_filename}'")

if __name__ == "__main__":
    main()

Created directory: ./results-per-position
Created 9 contest datasets

Dataset: PROVINCIAL VICE-GOVERNOR of PANGASINAN
Shape: 2 candidates × 88 columns
                  Candidate  abanon  agdao  anando  ano  antipangol  aponit  \
0  1. LAMBINO, MARK (LAKAS)     742   1292     360  960         593     791   
1      2. NACAR, NOEL (API)     201    462      95  351         267     204   

   bacnar  balaya  balayong  ...  talang  tamayo  tandang-sora  tandoc  \
0    1780     684       696  ...     669     910           251    1150   
1     602     229       213  ...     160     337            88     406   

   tarece  tarectec  tayambani  tebag  turac  Total  
0    1468       637        483    455   1861  67244  
1     411       163        136    183    665  22345  

[2 rows x 88 columns]


Saved to ./results-per-position/PROVINCIAL_VICEGOVERNOR_of_PANGASINAN_results.csv


Dataset: VICE-MAYOR of PANGASINAN - CITY OF SAN CARLOS
Shape: 2 candidates × 88 columns
                   Candidate 

In [5]:
def create_mayor_graphs(results_folder="./results-per-position"):
    """
    Create line and bar graphs showing mayoral votes per barangay
    """
    # Use the exact filename from the screenshot
    mayor_file = os.path.join(results_folder, "MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_results.csv")
    
    # Check if the file exists
    if not os.path.exists(mayor_file):
        print(f"Error: File does not exist: {mayor_file}")
        # List all files in the directory to debug
        print("Files in the directory:")
        for file in os.listdir(results_folder):
            print(f"  - {file}")
        return
    
    print(f"Found mayor file: {mayor_file}")
    contest_name = "MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS"
    
    # Load the mayor data
    mayor_df = pd.read_csv(mayor_file)
    
    # Get the candidate and barangay columns
    candidates = mayor_df['Candidate'].tolist()
    
    # Simplify candidate names for better display (just get the name part)
    simplified_names = []
    for name in candidates:
        # Extract the name part between the number and the parenthesis
        match = re.search(r'\d+\.\s+(.*?)\s+\(', name)
        if match:
            simplified_names.append(match.group(1))
        else:
            simplified_names.append(name)
    
    # Get barangay columns (exclude 'Candidate' and 'Total')
    barangays = [col for col in mayor_df.columns if col not in ['Candidate', 'Total']]
    
    # Transpose the data for plotting (exclude 'Candidate' and 'Total' columns)
    plot_data = mayor_df[barangays].copy()
    plot_data.index = simplified_names
    
    # Create directory for graphs if it doesn't exist
    graphs_folder = os.path.join(results_folder, 'graphs')
    if not os.path.exists(graphs_folder):
        os.makedirs(graphs_folder)
        print(f"Created directory: {graphs_folder}")
    
    # Generate a clean name for filenames
    clean_contest = re.sub(r'[^\w\s]', '', contest_name).replace(' ', '_')
    
    # ========== LINE GRAPH ==========
    plt.figure(figsize=(14, 8))
    
    # Plot each candidate as a line
    for i, candidate in enumerate(simplified_names):
        plt.plot(barangays, plot_data.loc[candidate], marker='o', linewidth=2, 
                 label=candidate, markersize=8)
    
    # Add labels and title
    plt.title('Mayoral Votes per Barangay (Line Graph)', fontsize=16)
    plt.xlabel('Barangay', fontsize=14)
    plt.ylabel('Number of Votes', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.legend(title='Candidates', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Adjust layout and save
    plt.tight_layout()
    line_graph_path = os.path.join(graphs_folder, f'{clean_contest}_line_graph.png')
    plt.savefig(line_graph_path, dpi=300)
    plt.close()
    print(f"Saved line graph to: {line_graph_path}")
    
    # ========== BAR GRAPH ==========
    plt.figure(figsize=(14, 8))
    
    # Set the positions for the bars
    bar_width = 0.8 / len(candidates)
    positions = np.arange(len(barangays))
    
    # Plot each candidate as a group of bars
    for i, candidate in enumerate(simplified_names):
        offset = i * bar_width - (len(candidates) * bar_width / 2) + (bar_width / 2)
        bars = plt.bar(positions + offset, plot_data.loc[candidate], 
                        width=bar_width, label=candidate)
    
    # Add labels and title
    plt.title('Mayoral Votes per Barangay (Bar Graph)', fontsize=16)
    plt.xlabel('Barangay', fontsize=14)
    plt.ylabel('Number of Votes', fontsize=14)
    plt.xticks(positions, barangays, rotation=45, ha='right', fontsize=12)
    plt.legend(title='Candidates', fontsize=12)
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    # Adjust layout and save
    plt.tight_layout()
    bar_graph_path = os.path.join(graphs_folder, f'{clean_contest}_bar_graph.png')
    plt.savefig(bar_graph_path, dpi=300)
    plt.close()
    print(f"Saved bar graph to: {bar_graph_path}")
    
    # ========== GROUPED BAR GRAPH (Alternative View) ==========
    plt.figure(figsize=(14, 8))
    
    # Transpose for grouped bar chart
    stacked_data = plot_data.T
    
    # Create grouped bar chart
    stacked_data.plot(kind='bar', stacked=False, figsize=(14, 8))
    
    # Add labels and title
    plt.title('Mayoral Votes per Barangay (Grouped Bar Graph)', fontsize=16)
    plt.xlabel('Barangay', fontsize=14)
    plt.ylabel('Number of Votes', fontsize=14)
    plt.xticks(rotation=45, ha='right', fontsize=12)
    plt.legend(title='Candidates', fontsize=12)
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    # Adjust layout and save
    plt.tight_layout()
    grouped_bar_path = os.path.join(graphs_folder, f'{clean_contest}_grouped_bar_graph.png')
    plt.savefig(grouped_bar_path, dpi=300)
    plt.close()
    print(f"Saved grouped bar graph to: {grouped_bar_path}")

# Create the graphs
create_mayor_graphs("./results-per-position")

Found mayor file: ./results-per-position/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_results.csv
Created directory: ./results-per-position/graphs
Saved line graph to: ./results-per-position/graphs/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph.png
Saved bar graph to: ./results-per-position/graphs/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_bar_graph.png
Saved grouped bar graph to: ./results-per-position/graphs/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_grouped_bar_graph.png


<Figure size 1400x800 with 0 Axes>

In [13]:
def create_improved_mayor_graphs(results_folder="./results-per-position"):
    """
    Create improved visualizations for mayoral votes with better handling of many barangays
    """
    
    # Use the exact filename from the original code
    mayor_file = os.path.join(results_folder, "MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_results.csv")
    
    # Check if the file exists
    if not os.path.exists(mayor_file):
        print(f"Error: File does not exist: {mayor_file}")
        # List all files in the directory to debug
        print("Files in the directory:")
        for file in os.listdir(results_folder):
            print(f"  - {file}")
        return
    
    print(f"Found mayor file: {mayor_file}")
    contest_name = "MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS"
    
    # Load the mayor data
    mayor_df = pd.read_csv(mayor_file)
    
    # Get the candidate and barangay columns
    candidates = mayor_df['Candidate'].tolist()
    
    # Simplify candidate names for better display
    simplified_names = []
    for name in candidates:
        # Extract the name part between the number and the parenthesis
        match = re.search(r'\d+\.\s+(.*?)\s+\(', name)
        if match:
            simplified_names.append(match.group(1))
        else:
            simplified_names.append(name)
    
    # Get barangay columns (exclude 'Candidate' and 'Total')
    barangays = [col for col in mayor_df.columns if col not in ['Candidate', 'Total']]
    
    # Transpose the data for plotting
    plot_data = mayor_df[barangays].copy()
    plot_data.index = simplified_names
    
    # Create directory for graphs in the root directory
    graphs_folder = "./mayor_visualizations"
    if not os.path.exists(graphs_folder):
        os.makedirs(graphs_folder)
        print(f"Created directory: {graphs_folder}")
    
    # Generate a clean name for filenames
    clean_contest = re.sub(r'[^\w\s]', '', contest_name).replace(' ', '_')
    
    # ========== SOLUTION 1: SPLIT INTO MULTIPLE GRAPHS ==========
    # Define how many barangays per graph
    barangays_per_graph = 15
    
    # Calculate how many graphs we need
    num_graphs = len(barangays) // barangays_per_graph
    if len(barangays) % barangays_per_graph > 0:
        num_graphs += 1
    
    # Create a line graph for each group of barangays
    for graph_num in range(num_graphs):
        start_idx = graph_num * barangays_per_graph
        end_idx = min(start_idx + barangays_per_graph, len(barangays))
        
        current_barangays = barangays[start_idx:end_idx]
        
        plt.figure(figsize=(14, 8))
        
        # Plot each candidate as a line
        for i, candidate in enumerate(simplified_names):
            plt.plot(current_barangays, plot_data.loc[candidate, current_barangays], 
                     marker='o', linewidth=2, label=candidate, markersize=8)
        
        # Add labels and title
        plt.title(f'Mayoral Votes per Barangay - Group {graph_num+1} (Line Graph)', fontsize=16)
        plt.xlabel('Barangay', fontsize=14)
        plt.ylabel('Number of Votes', fontsize=14)
        plt.xticks(rotation=45, ha='right', fontsize=12)
        plt.legend(title='Candidates', fontsize=12)
        plt.grid(True, linestyle='--', alpha=0.7)
        
        # Adjust layout and save
        plt.tight_layout()
        split_line_path = os.path.join(graphs_folder, f'{clean_contest}_line_graph_group_{graph_num+1}.png')
        plt.savefig(split_line_path, dpi=300)
        plt.close()
        print(f"Saved split line graph {graph_num+1} to: {split_line_path}")
    
    # ========== SOLUTION 2: HEATMAP VISUALIZATION ==========
    plt.figure(figsize=(16, 10))
    
    # Normalize the data for color mapping
    norm = Normalize(vmin=plot_data.values.min(), vmax=plot_data.values.max())
    
    # Plot the heatmap
    plt.imshow(plot_data, aspect='auto', cmap='viridis', norm=norm)
    
    # Add labels and title
    plt.title('Mayoral Votes per Barangay (Heatmap)', fontsize=16)
    plt.xlabel('Barangay', fontsize=14)
    plt.ylabel('Candidate', fontsize=14)
    
    # Add x and y ticks
    plt.xticks(np.arange(len(barangays)), barangays, rotation=90, fontsize=8)
    plt.yticks(np.arange(len(simplified_names)), simplified_names, fontsize=12)
    
    # Add a colorbar
    cbar = plt.colorbar()
    cbar.set_label('Number of Votes', fontsize=14)
    
    # Adjust layout and save
    plt.tight_layout()
    heatmap_path = os.path.join(graphs_folder, f'{clean_contest}_heatmap.png')
    plt.savefig(heatmap_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved heatmap to: {heatmap_path}")
    
    # ========== SOLUTION 3: TOP BARANGAYS BAR CHART ==========
    # Calculate total votes per barangay
    barangay_totals = plot_data.sum()
    
    # Get the top 15 barangays by total votes
    top_barangays = barangay_totals.nlargest(15).index.tolist()
    
    plt.figure(figsize=(14, 8))
    
    # Set the positions for the bars
    bar_width = 0.8 / len(simplified_names)
    positions = np.arange(len(top_barangays))
    
    # Plot each candidate as a group of bars, but only for top barangays
    for i, candidate in enumerate(simplified_names):
        offset = i * bar_width - (len(simplified_names) * bar_width / 2) + (bar_width / 2)
        plt.bar(positions + offset, plot_data.loc[candidate, top_barangays], 
                width=bar_width, label=candidate)
    
    # Add labels and title
    plt.title('Mayoral Votes for Top 15 Barangays by Total Votes', fontsize=16)
    plt.xlabel('Barangay', fontsize=14)
    plt.ylabel('Number of Votes', fontsize=14)
    plt.xticks(positions, top_barangays, rotation=45, ha='right', fontsize=12)
    plt.legend(title='Candidates', fontsize=12)
    plt.grid(True, axis='y', linestyle='--', alpha=0.7)
    
    # Adjust layout and save
    plt.tight_layout()
    top_barangays_path = os.path.join(graphs_folder, f'{clean_contest}_top_barangays.png')
    plt.savefig(top_barangays_path, dpi=300)
    plt.close()
    print(f"Saved top barangays chart to: {top_barangays_path}")
    
    # ========== SOLUTION 4: INTERACTIVE PLOTLY VISUALIZATION ==========
    # Prepare data for Plotly
    plotly_data = []
    
    # Convert the data to long format for Plotly
    long_data = plot_data.reset_index().melt(
        id_vars='index', 
        value_vars=barangays,
        var_name='Barangay', 
        value_name='Votes'
    )
    long_data.rename(columns={'index': 'Candidate'}, inplace=True)
    
    # Create an interactive line chart
    fig = px.line(
        long_data, 
        x='Barangay', 
        y='Votes', 
        color='Candidate',
        title='Interactive Mayoral Votes per Barangay',
        labels={'Votes': 'Number of Votes', 'Barangay': 'Barangay'},
        height=600,
        markers=True
    )
    
    # Update layout for better readability
    fig.update_layout(
        xaxis_title='Barangay',
        yaxis_title='Number of Votes',
        legend_title='Candidates',
        xaxis={'tickangle': 45},
        hovermode='closest'
    )
    
    # Save as HTML
    interactive_path = os.path.join(graphs_folder, f'{clean_contest}_interactive.html')
    fig.write_html(interactive_path)
    print(f"Saved interactive visualization to: {interactive_path}")
    
    # ========== SOLUTION 5: CANDIDATE COMPARISON BAR CHART ==========
    # Calculate total votes per candidate
    candidate_totals = plot_data.sum(axis=1).sort_values(ascending=False)
    
    plt.figure(figsize=(12, 8))
    
    # Create horizontal bar chart of total votes per candidate
    bars = plt.barh(np.arange(len(candidate_totals)), candidate_totals.values, height=0.5)
    
    # Add labels to the bars
    for i, bar in enumerate(bars):
        plt.text(bar.get_width() + 500, bar.get_y() + bar.get_height()/2, 
                 f'{candidate_totals.values[i]:,}', 
                 va='center', fontsize=12)
    
    # Add labels and title
    plt.title('Total Votes per Mayoral Candidate', fontsize=16)
    plt.xlabel('Number of Votes', fontsize=14)
    plt.ylabel('Candidate', fontsize=14)
    plt.yticks(np.arange(len(candidate_totals)), candidate_totals.index)
    plt.grid(True, axis='x', linestyle='--', alpha=0.7)
    
    # Adjust layout and save
    plt.tight_layout()
    candidate_totals_path = os.path.join(graphs_folder, f'{clean_contest}_candidate_totals.png')
    plt.savefig(candidate_totals_path, dpi=300)
    plt.close()
    print(f"Saved candidate totals chart to: {candidate_totals_path}")
    
    print("All improved visualizations created successfully!")

# Run the function
create_improved_mayor_graphs("./results-per-position")

Found mayor file: ./results-per-position/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_results.csv
Saved split line graph 1 to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph_group_1.png
Saved split line graph 2 to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph_group_2.png
Saved split line graph 3 to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph_group_3.png
Saved split line graph 4 to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph_group_4.png
Saved split line graph 5 to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph_group_5.png
Saved split line graph 6 to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_line_graph_group_6.png
Saved heatmap to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_heatmap.png
Saved top barangays chart to: ./mayor_visualizations/MAYOR_of_PANGASINAN__CITY_OF_SAN_CARLOS_top_barangays.png
Saved interac

In [21]:
# Set up visualization directory
image_dir = Path("./election_visualizations_senator")
image_dir.mkdir(exist_ok=True)
print(f"Created visualization directory: {image_dir.absolute()}")

# Set matplotlib style
plt.style.use('seaborn-v0_8-whitegrid')  # Back to whitegrid (not pure white, has a light gray grid)
sns.set_palette("dark")  # Keep the dark palette for more saturated colors

# Set global figure size
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['savefig.dpi'] = 300  # High resolution for saved images

# Load the data
print("Loading election data...")
filepath = './results-per-position/SENATOR_of_PHILIPPINES_results.csv'
df = pd.read_csv(filepath)

print(f"Dataset shape: {df.shape}")
print(f"Number of candidates: {df.shape[0]}")

# Data Preparation
print("Preparing data for analysis...")
data = df.copy()

# Extract candidate names (remove numbers and party affiliations)
data['Clean_Name'] = data['Candidate'].str.extract(r'^\d+\.\s*(.+?)(?:\s*\([^)]+\))?$')

# Identify location columns (all columns except Candidate and Total)
location_cols = [col for col in data.columns if col not in ['Candidate', 'Clean_Name', 'Total']]

# Sort candidates by total votes
data_sorted = data.sort_values('Total', ascending=False).reset_index(drop=True)

# Display the top 15 candidates
print("\nTop 15 candidates by votes:")
top_15 = data_sorted[['Candidate', 'Clean_Name', 'Total']].head(15)
print(top_15.to_string(index=False))

# Save the top 15 candidates to CSV
top_15.to_csv(image_dir / "top_15_candidates.csv", index=False)
print(f"Saved: {image_dir / 'top_15_candidates.csv'}")

# Visualization 1: Top 15 Candidates by Total Votes
print("\nCreating visualization of top 15 candidates by votes...")

# Plot top 15 candidates
plt.figure(figsize=(12, 8))
top_15_plot = data_sorted.head(15).sort_values('Total')
# Changed to BuPu (blue to purple) - avoids yellow and pure white
bars = plt.barh(top_15_plot['Clean_Name'], top_15_plot['Total'], 
                color=plt.cm.BuPu(np.linspace(0.4, 0.9, 15)))  # Using range 0.4-0.9 to avoid too light colors

plt.title('Top 15 Senatorial Candidates by Total Votes', fontsize=16)
plt.xlabel('Total Votes', fontsize=12)
plt.ylabel('Candidate', fontsize=12)

# Add vote counts as labels
for i, bar in enumerate(bars):
    plt.text(bar.get_width() + 500, bar.get_y() + bar.get_height()/2, 
             f"{top_15_plot['Total'].iloc[i]:,}", 
             va='center', fontsize=10)

plt.tight_layout()

# Save the matplotlib figure
top_candidates_filepath = image_dir / "top_15_candidates.png"
plt.savefig(top_candidates_filepath, bbox_inches='tight')
print(f"Saved: {top_candidates_filepath}")
plt.close()

# Visualization 2: Top Barangays for #1 Candidate
print("\nAnalyzing top barangays for the #1 candidate...")

# Get the top candidate
top_candidate = data_sorted.iloc[0]
top_candidate_name = top_candidate['Clean_Name']
print(f"#1 Candidate: {top_candidate_name}")

# Create a dataframe of location votes for the top candidate
location_votes = pd.DataFrame({
    'Barangay': [loc.replace('-', ' ').title() for loc in location_cols],
    'Votes': [top_candidate[loc] for loc in location_cols]
})

# Sort by votes
location_votes = location_votes.sort_values('Votes', ascending=False).reset_index(drop=True)

# Display top 20 barangays
print("\nTop 20 barangays for the #1 candidate:")
print(location_votes.head(20).to_string(index=False))

# Save top barangays to CSV
location_votes.head(20).to_csv(image_dir / f"top_barangays_for_{top_candidate_name}.csv", index=False)
print(f"Saved: {image_dir / f'top_barangays_for_{top_candidate_name}.csv'}")

# Plot top 20 barangays
plt.figure(figsize=(14, 10))
top_20_barangays = location_votes.head(20).sort_values('Votes')
# Changed to BuPu (blue to purple) - avoids yellow and pure white
bars = plt.barh(top_20_barangays['Barangay'], top_20_barangays['Votes'], 
        color=plt.cm.BuPu(np.linspace(0.4, 0.9, 20)))  # Using range 0.4-0.9 to avoid too light colors

plt.title(f'Top 20 Barangays for {top_candidate_name}', fontsize=16)
plt.xlabel('Votes', fontsize=12)
plt.ylabel('Barangay', fontsize=12)

# Add vote counts as labels
for i, bar in enumerate(bars):
    plt.text(bar.get_width() + 20, bar.get_y() + bar.get_height()/2, 
             f"{top_20_barangays['Votes'].iloc[i]:,}", 
             va='center', fontsize=10)

plt.tight_layout()

# Save the matplotlib figure
top_barangays_filepath = image_dir / f"top_20_barangays_for_{top_candidate_name}.png"
plt.savefig(top_barangays_filepath, bbox_inches='tight')
print(f"Saved: {top_barangays_filepath}")
plt.close()

print("\nAnalysis complete!")
print(f"All visualizations and data have been saved to: {image_dir.absolute()}")

Created visualization directory: /home/ray/Desktop/local-election-san-carlos-analysis/election_visualizations_senator
Loading election data...
Dataset shape: (66, 88)
Number of candidates: 66
Preparing data for analysis...

Top 15 candidates by votes:
                         Candidate              Clean_Name  Total
              5. AQUINO, BAM (KNP)             AQUINO, BAM  51346
             59. SOTTO, TITO (NPC)             SOTTO, TITO  45514
          63. TULFO, ERWIN (LAKAS)            TULFO, ERWIN  44365
           1. ABALOS, BENHUR (PFP)          ABALOS, BENHUR  41272
            18. CAYETANO, PIA (NP)           CAYETANO, PIA  40018
             35. LAPID, LITO (NPC)             LAPID, LITO  38660
         51. PANGILINAN, KIKO (LP)        PANGILINAN, KIKO  38253
       28. GO, BONG GO    (PDPLBN)             GO, BONG GO  37206
            33. LACSON, PING (IND)            LACSON, PING  36612
        62. TULFO, BEN BITAG (IND)        TULFO, BEN BITAG  33661
11. BONG REVILLA,RAMON